In [1]:
!pip install transformers

In [2]:
from google.colab import drive
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
import pandas as pd
import numpy as np

In [3]:
MODEL_NAME = 'rdenadai/BR_BERTo'
DATA_PATH = '/content/drive/My Drive/TCC/basededadosUTF8.csv'

In [4]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
df = df = pd.read_csv(DATA_PATH)

In [6]:
all_labels = df['Categoria'].astype(int).to_numpy()
all_texts = []
for text in df['Texto']:
  all_texts.append(text)

In [7]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

In [8]:
texts_encodings = tokenizer(all_texts, padding='max_length', truncation=True,max_length=380, return_tensors="pt")

In [9]:
class FakeBRTwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = FakeBRTwitterDataset(texts_encodings, all_labels)

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    mcc = matthews_corrcoef(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'mcc':mcc
    }

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=7,              # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=50,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    evaluation_strategy='no',
    logging_dir='./logs',            # directory for storing logs
)

In [11]:
kf = KFold(n_splits=10,shuffle=True)

In [12]:
results = []
for train_index, test_index in kf.split(dataset):
  train_dataset = []
  for index in train_index:
    train_dataset.append(dataset[index])
  test_dataset = []
  for index in test_index:
    test_dataset.append(dataset[index])
  model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME,return_dict=True)
  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
  )
  trainer.train()
  eval_result = trainer.evaluate()
  results.append(eval_result)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
Some weights of the model checkpoint at rdenadai/BR_BERTo were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you exp

Step,Training Loss


Some weights of the model checkpoint at rdenadai/BR_BERTo were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at rdenadai/BR_BERTo and are newly initialized: ['cl

Step,Training Loss


Some weights of the model checkpoint at rdenadai/BR_BERTo were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at rdenadai/BR_BERTo and are newly initialized: ['cl

...

Step,Training Loss


Some weights of the model checkpoint at rdenadai/BR_BERTo were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at rdenadai/BR_BERTo and are newly initialized: ['cl

Step,Training Loss


Some weights of the model checkpoint at rdenadai/BR_BERTo were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at rdenadai/BR_BERTo and are newly initialized: ['cl

Step,Training Loss


In [13]:
acc = []
f1 = []
precision = []
recall = []
mcc=[]
for result in results:
  acc.append(result['eval_accuracy'])
  f1.append(result['eval_f1'])
  precision.append(result['eval_precision'])
  recall.append(result['eval_recall'])
  mcc.append(result['eval_mcc'])
print(np.mean(acc))
print(np.mean(f1)) 
print(np.mean(precision))
print(np.mean(recall))
print(np.mean(mcc))

0.8595588235294118
0.8726259489302967
0.8276948051948052
0.9334415584415584
0.7263547321827382
